# Configure the PayPal SDK client

In [ ]:
# Load credentials from environment (as set earlier)
client_id = ""
client_secret = ""


In [3]:
import os
import requests
from requests.auth import HTTPBasicAuth

# PayPal sandbox OAuth endpoint
token_url = "https://api-m.sandbox.paypal.com/v1/oauth2/token"

# Request an access token
resp = requests.post(
    token_url,
    auth=HTTPBasicAuth(client_id, client_secret),
    headers={"Accept": "application/json"},
    data={"grant_type": "client_credentials"}
)
resp.raise_for_status()
access_token = resp.json()["access_token"]

print("✅ Access token:", access_token)


✅ Access token: A21AAKJmNE7ixQArhcvsuihjpyi1i3NvC3dddFBP3LLN0kQ6T4NLCniio46dprqjvR5RTdl4fdQ2KFh3-m4YfPp0lTeGkgKLg


# Create a one-time PayPal order

In [15]:
import requests
import os

# Step 1: PayPal Orders API endpoint (Sandbox)
order_url = "https://api-m.sandbox.paypal.com/v2/checkout/orders"

# Step 2: HTTP headers for auth
headers = {
    "Content-Type": "application/json",             # JSON payload
    "Authorization": f"Bearer {access_token}"       # Your OAuth token
}

# Step 3: Order payload
payload = {
    "intent": "CAPTURE",                            # Auto-capture once approved
    "purchase_units": [
        {
            "amount": {
                "currency_code": "USD",            # USD transaction
                "value": "10.00"                    # $10 one-time fee
            },
            "description": "One-time $10 subscription to ChirpChecker"
        }
    ],
    "application_context": {
        "brand_name": "ChirpChecker",               # Your app name in the PayPal UI
        "landing_page": "NO_PREFERENCE",            # Let PayPal decide best layout
        "user_action": "PAY_NOW",                   # Button reads “Pay Now”
        "return_url": "http://localhost:3000/return",  # After approval, PayPal redirects here
        "cancel_url": "http://localhost:3000/cancel"   # If canceled, user returns here
    }
}

# Step 4: Create the order
response = requests.post(order_url, headers=headers, json=payload)
response.raise_for_status()                         # Error if non-2xx
order = response.json()

# Step 5: Pull out the approval link
approve_link = next(
    (link["href"] for link in order["links"] if link["rel"] == "approve"),
    None
)

# Output for you to use
print("✅ Order ID:", order["id"])
print("👉 Approval URL:", approve_link)


✅ Order ID: 57144656TN7503030
👉 Approval URL: https://www.sandbox.paypal.com/checkoutnow?token=57144656TN7503030


# Capture the approved order

In [22]:
import requests

order_id = "57144656TN7503030"   # your actual order ID
capture_url = f"https://api-m.sandbox.paypal.com/v2/checkout/orders/{order_id}/capture"

try:
    resp = requests.post(capture_url, headers=headers)
    resp.raise_for_status()
    data = resp.json()
    print("✅ Capture succeeded:", data)
except requests.HTTPError:
    print("❌ Capture failed with status:", resp.status_code)
    # Print the full error details from PayPal
    print(resp.json())


❌ Capture failed with status: 422
{'name': 'UNPROCESSABLE_ENTITY', 'details': [{'issue': 'ORDER_NOT_APPROVED', 'description': "Payer has not yet approved the Order for payment. Please redirect the payer to the 'rel':'approve' url returned as part of the HATEOAS links within the Create Order call or provide a valid payment_source in the request."}], 'message': 'The requested action could not be performed, semantically incorrect, or failed business validation.', 'debug_id': 'f2572602e5a93', 'links': [{'href': 'https://developer.paypal.com/api/rest/reference/orders/v2/errors/#ORDER_NOT_APPROVED', 'rel': 'information_link', 'method': 'GET'}]}


# Paypal Subscription

In [29]:
import os, requests
from requests.auth import HTTPBasicAuth

# ————————————————
# CONFIGURATION
# ————————————————
# Your sandbox credentials (make sure these are set in your env or replace directly)
CLIENT_ID     = os.getenv("PAYPAL_CLIENT_ID")
CLIENT_SECRET = os.getenv("PAYPAL_CLIENT_SECRET")

# Replace these with the actual plan IDs from your PayPal sandbox
MONTHLY_PLAN_ID = "P-XXXXXXXXXXXXXXXXX"  # $10/month plan
YEARLY_PLAN_ID  = "P-YYYYYYYYYYYYYYY"  # $100/year plan

# API endpoints
OAUTH_URL         = "https://api-m.sandbox.paypal.com/v1/oauth2/token"
ORDER_URL         = "https://api-m.sandbox.paypal.com/v2/checkout/orders"
SUBSCRIPTION_URL  = "https://api-m.sandbox.paypal.com/v1/billing/subscriptions"


# ————————————————
# HELPERS
# ————————————————
def get_access_token():
    """
    Fetch a sandbox OAuth2 token using client credentials.
    """
    resp = requests.post(
        OAUTH_URL,
        auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET),
        data={"grant_type": "client_credentials"}
    )
    resp.raise_for_status()
    return resp.json()["access_token"]


def create_payment(plan: str, scheme: str):
    """
    plan:   "monthly" or "yearly"
    scheme: "one_time" or "recurring"
    
    Returns (id, approval_url)
    """
    token = get_access_token()
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }

    if scheme == "one_time":
        # — Build a one-time order for $10 or $100
        amount = "10.00" if plan == "monthly" else "100.00"
        payload = {
            "intent": "CAPTURE",
            "purchase_units": [
                {"amount": {"currency_code": "USD", "value": amount}}
            ],
            "application_context": {
                "brand_name": "ChirpChecker",
                "landing_page": "NO_PREFERENCE",
                "user_action": "PAY_NOW",
                "return_url": "http://localhost:3000/return",
                "cancel_url": "http://localhost:3000/cancel"
            }
        }
        resp = requests.post(ORDER_URL, headers=headers, json=payload)
        resp.raise_for_status()
        data = resp.json()
        order_id = data["id"]
        approve_url = next(l["href"] for l in data["links"] if l["rel"] == "approve")
        return order_id, approve_url

    elif scheme == "recurring":
        # — Create a subscription for the chosen plan ID
        plan_id = MONTHLY_PLAN_ID if plan == "monthly" else YEARLY_PLAN_ID
        payload = {
            "plan_id": plan_id,
            "application_context": {
                "brand_name": "ChirpChecker",
                "return_url": "http://localhost:3000/return",
                "cancel_url": "http://localhost:3000/cancel"
            }
        }
        resp = requests.post(SUBSCRIPTION_URL, headers=headers, json=payload)
        resp.raise_for_status()
        data = resp.json()
        sub_id = data["id"]
        approve_url = next(l["href"] for l in data["links"] if l["rel"] == "approve")
        return sub_id, approve_url

    else:
        raise ValueError("scheme must be 'one_time' or 'recurring'")


def check_status(entity_id: str, scheme: str):
    """
    Fetches the latest status of an order or subscription.
    scheme: "one_time" => Order; "recurring" => Subscription
    Returns the full JSON response (inspect ['status']).
    """
    token = get_access_token()
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }

    if scheme == "one_time":
        url = f"{ORDER_URL}/{entity_id}"
    elif scheme == "recurring":
        url = f"{SUBSCRIPTION_URL}/{entity_id}"
    else:
        raise ValueError("scheme must be 'one_time' or 'recurring'")

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()


# How to use

In [33]:
# 1) One-time $10 charge
order_id, url = create_payment("monthly", "one_time")
print("Order ID:", order_id)
print("Buyer approval URL:", url)

# 2) Recurring monthly subscription
sub_id, url = create_payment("monthly", "recurring")
print("Subscription ID:", sub_id)
print("Buyer approval URL:", url)

# 3) Later, check status
status = check_status(order_id, "one_time")
print("One-time order status:", status["status"])

sub_status = check_status(sub_id, "recurring")
print("Subscription status:", sub_status["status"])
